In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename)) # prints the full path of each file
import tensorflow as tf
import tensorflow_decision_forests as tfdf

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


**Load DataSet**

In [13]:
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")

train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#Prepare dataset:
#Tokenize the name
#"Braund, Mr. Owen Harris" will become ["Braund", "Mr.", "Owen", "Harris"]
#Extract any prefix ticket
#"STON/O2. 3101282" will become "STON/O2. and 310282

In [4]:
#function that takes in a dataframe df as input
def preprocess(df):

    #take copy of data
    df = df.copy()

    #strip special characters, join in single string w spaces
    def normalize_name(x):
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])

    #extracts last part of ticket
    def ticket_number(x):
        return x.split(" ")[-1]
    #extracts first part of ticket, in not then 'none'
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])

    #new dataframe
    df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)                     
    return df

#run preproccess function above on training and testing functions
preprocessed_train_df = preprocess(train_df)
preprocessed_serving_df = preprocess(serving_df)

preprocessed_train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,Heikkinen Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C123,S,113803,NONE
4,5,0,3,Allen Mr William Henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,NONE


In [5]:
#remove some features: we dont want train model on passenegrID,and ticket
input_features = list(preprocessed_train_df.columns)
input_features.remove("Ticket")
input_features.remove("PassengerId")
input_features.remove("Survived")
#input_features.remove("Ticket_number")

print(f"Input features: {input_features}")

Input features: ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']


In [6]:
def tokenize_names(features, labels=None):
    #Divde the names into individual tokens
    features["Name"] =  tf.strings.split(features["Name"])
    return features, labels

#tfdf.keras.pd_dataframe_to_tf_dataset : converts pandas df to tf df
#label = survived : Specifies the label column for supervised learning
#Applies the tokenize_names function to each row in the dataset
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df,label="Survived").map(tokenize_names)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(tokenize_names)

In [7]:
#First training a GradientBoostedTreesModel model with the default parameterst


#initialize model

#keras.GradientBoostedTreesModel: Ensemble learning algorithm combining decision trees
#works by iteratively building trees to minimize the error of previous trees
model = tfdf.keras.GradientBoostedTreesModel(
    
    #Reduces amount of logging informationfor simplicity
    verbose=0,
    
    #Specifies the features to use for training
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    #other features ignored
    exclude_non_specified_features=True,
    #Sets a fixed random seed for reproducibility
    random_seed=1234,
)

#Trains the Gradient Boosted Trees model on the training dataset 
model.fit(train_ds)

#Evaluates the model's performance using the training dataset and prints
self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

[WARNING 24-11-24 16:10:31.8452 UTC gradient_boosted_trees.cc:1840] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-11-24 16:10:31.8461 UTC gradient_boosted_trees.cc:1851] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-11-24 16:10:31.8461 UTC gradient_boosted_trees.cc:1865] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-11-24 16:10:36.9330 UTC kernel.cc:1233] Loading model from path /tmp/tmp1jhfyyh1/model/ with prefix 1115d0d7150c4a6e
[INFO 24-11-24 16:10:36.9389 UTC quick_scorer_extended.cc:911] The binary was compiled without AVX2 support, but your CPU supports it. Enable it for faster model inference.
[INFO 24-11-24 16:10:36.9394 UTC abstract_model.cc:1362] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 24-11-24 16:10:36.9394 UTC kernel.cc:1061] Use fast generic engine


Accuracy: 0.8260869383811951 Loss:0.8608942627906799


In [8]:
#Train model with improved default parameters

model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, 
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True,
    
    #Enables computation of permutation-based feature importance
    #slower but provides a deep understanding of the feature contributions
    #compute_permutation_variable_importance=True,

    # Uncommenting this could override the manual parameters
    # Change the default hyper-parameters
    # hyperparameter_template="benchmark_rank1@v1",
    
    #num_trees=1000,
    #tuner=tuner

    # Sets the min number of examples node for further splitting. Lower values more granular splits
    min_examples=1,

    #selects splits based on randomized subsets of categories
    categorical_algorithm="RANDOM",

    #Limits the depth of the trees to control overfitting
    #max_depth=4,
    
    #aka learning rate: contribution of each tree to final prediction. Smaller = more accurate, slower training
    shrinkage=0.05,

    #proportion of features considered for each split, reduces training time
    #num_candidate_attributes_ratio=0.2,

    #creates splits that involve linear combinations of features. To handle high-dimensional data effectively
    split_axis="SPARSE_OBLIQUE",
    
    #normalize data using min_max metho data before splitting
    sparse_oblique_normalization="MIN_MAX",
    
    #higher val = complex splits,capture more subtle patterns
    sparse_oblique_num_projections_exponent=2.0,

    #number of trees in ensemble, incr accuracy
    num_trees=2000,

    #ratio of training data reserved for validation
    #validation_ratio=0.0,
    
    random_seed=1234,
    )

#train model
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

[WARNING 24-11-24 16:10:42.4484 UTC gradient_boosted_trees.cc:1840] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-11-24 16:10:42.4484 UTC gradient_boosted_trees.cc:1851] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-11-24 16:10:42.4484 UTC gradient_boosted_trees.cc:1865] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-11-24 16:10:43.4236 UTC kernel.cc:1233] Loading model from path /tmp/tmpf0d9557h/model/ with prefix d2689b95109f4412
[INFO 24-11-24 16:10:43.4316 UTC decision_forest.cc:734] Model loaded with 40 root(s), 2106 node(s), and 10 input feature(s).
[INFO 24-11-24 16:10:43.4316 UTC kernel.cc:1061] Use fast generic engine


Accuracy: 0.782608687877655 Loss:1.0586705207824707


In [9]:
model.summary()

Model: "gradient_boosted_trees_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	Age
	Cabin
	Embarked
	Fare
	Name
	Parch
	Pclass
	Sex
	SibSp
	Ticket_item
	Ticket_number

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.           "Sex"  0.585997 ################
    2.           "Age"  0.364636 #######
    3.          "Fare"  0.266191 ###
    4.          "Name"  0.207054 #
    5.        "Pclass"  0.179191 
    6. "Ticket_number"  0.178806 
    7.      "Embarked"  0.177803 
    8.   "Ticket_item"  0.177009 
    9.         "Parch"  0.175276 
   10.         "SibSp"  0.171694 

Variable Importance: NUM_AS_ROOT:
    1.  "Sex" 34.000000 ######

In [14]:
#Predictions
def prediction_format(model, threshold=0.5):
    pb_survive = model.predict(test_ds, verbose=0)[:,0]
    return pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        #cutoff probability = threshold = 0.5
        "Survived": (pb_survive >= threshold).astype(int)
    })
    
def make_submission(predictions):
    path="/kaggle/working/submission.csv"
    predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")

predictions = prediction_format(model)
make_submission(predictions)
!head /kaggle/working/submission.csv

Submission exported to /kaggle/working/submission.csv
PassengerId,Survived
892,0
893,0
894,0
895,0
896,0
897,0
898,0
899,0
900,1
